In [ ]:
gcs_video_files_directory="" #GCS URI to video files in format gs://bucket_name/path, files must be stored in subdirectories named after language code of the video
gcs_audio_files_directory="" #GCS URI to upload audio files

In [ ]:
!(sudo apt update && sudo apt install -y ffmpeg)

In [ ]:
import re
import os
from google.cloud import storage

storage_client = storage.Client()
video_bucket_name,video_path = re.search(f"gs://([^/]+)/(.*)",gcs_video_files_directory).group(1,2)
video_bucket = storage_client.bucket(video_bucket_name)
audio_bucket_name,audio_path = re.search(f"gs://([^/]+)/(.*)",gcs_audio_files_directory).group(1,2)
audio_bucket = storage_client.bucket(audio_bucket_name)

language_directories = !(gsutil ls {gcs_video_files_directory})
for language_directory in language_directories:
    print(f"Working on directory: {language_directory}")
    language_code = re.search("/([^/]+)/$",language_directory).group(1)
    if not os.path.exists(f"video/{language_code}/"):
        os.makedirs(f"video/{language_code}/")
    if not os.path.exists(f"audio/{language_code}/"):
        os.makedirs(f"audio/{language_code}/")
    uris=!(gsutil ls {language_directory})
    
    for uri in uris:
        print(f"Working on file: {uri}")
        source_path,source_filename=re.search("gs://[^/]+/(.*)/([^/]+)$",uri).group(1,2)
        
        video_blob=video_bucket.blob(f"{source_path}/{source_filename}")
        video_blob.download_to_filename(f"video/{language_code}/{source_filename}")
        
        destination_filename = re.search("([^.]+).",source_filename).group(1) + ".flac"
        
        !(ffmpeg -y -i video/{language_code}/{source_filename} -vn -f flac -ac 1 audio/{language_code}/{destination_filename})
        
        audio_blob=audio_bucket.blob(f"{audio_path}/{language_code}/{destination_filename}")
        audio_blob.upload_from_filename(f"audio/{language_code}/{destination_filename}")